# Ramp

In [2]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep conreg = { git = "https://github.com/almedso/conreg", default_features = false, features = ["std"] }

In [3]:
use conreg::regulation::ramp::{Ramp, GeneralizedPosition, RampConstraints};
use std::iter;
use plotters::prelude::*;
use plotters::prelude::LineSeries;

# Plot for position ramp

In [18]:
let sample_interval = 0.001_f32;  // in seconds
// Instanciate and configure the ramp
let mut r = Ramp::new(sample_interval,
    RampConstraints::default()
        .max_speed(2.0)
        .max_acceleration(1.0)
        .max_jerk(1.0)
    );
r.set(GeneralizedPosition::new(5.0));
r.set_target_relative_position(-5.0);
println!("Ramp: {:?}", r);

Ramp: Ramp { sample_interval: 0.001, constraints: RampConstraints { max_speed: Some(2.0), max_acceleration: Some(1.0), max_jerk: Some(1.0) }, current: GeneralizedPosition { position: 5.0, speed: 0.0, acceleration: 0.0 }, state: [AccelerateTo(-1.0), AccelerateConst(1.0), AccelerateTo(0.0), SpeedConst(1.0), AccelerateTo(1.0), AccelerateConst(1.0), AccelerateTo(0.0), SpeedZero] }


In [19]:
let time_range = 7.0_f32;  // in seconds
let i_range = (time_range / sample_interval ) as usize;
// Run the ramp 
let gp = r.take(i_range).collect::<Vec<GeneralizedPosition>>();

In [20]:
// Extract position, speed and acceleration from generalized parameters for plotting
let p = gp.clone().iter().map(|x| x.position).collect::<Vec<f32>>();
let s = gp.clone().iter().map(|x| x.speed).collect::<Vec<f32>>();
let a = gp.clone().iter().map(|x| x.acceleration).collect::<Vec<f32>>();

// Prepare the plot
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Position/speed/acceleration time diagram", ("Arial", 24).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0f32..time_range, -5.0f32..5.0f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(p),
        &GREEN,
    )).unwrap()
        .label("Position")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(s),
        &RED,
    )).unwrap()
        .label("Velocity")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(a),
        &BLUE,
    )).unwrap()
        .label("Acceleration")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperRight)
        .draw()?;
    Ok(())
});

In [21]:
// Show the ramp plot
figure

Position/speed/acceleration time diagram
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-4.0
 
 
 
-3.0
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 55,35 55,35 55,35 55,35 55,35 55,35 55,35 55,35 55,35 55,35 55,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 56,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 57,35 58,35 58,35 58,35 58,35 58,35 58,35 58,35 58,35 58,35 58,35 58,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 59,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 60,35 61,35 61,35 61,35 61,35 61,35 61,35 61,35 61,35 61,35 61,35 61,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 62,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 63,35 64,35 64,35 64,35 64,35 64,35 64,35 64,35 64,35 64,35 64,35 64,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 65,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 66,35 67,35 67,35 67,35 67,35 67,35 67,35 67,35 67,35 67,35 67,35 67,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 68,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 69,35 70,35 70,35 70,35 70,35 70,35 70,35 70,35 70,35 70,35 70,35 70,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 71,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 72,35 73,35 73,35 73,35 73,35 73,35 73,35 73,35 73,35 73,35 73,35 73,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 74,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 75,35 76,35 76,35 76,35 76,35 76,35 76,35 76,35 76,35 76,35 76,35 76,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 77,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 78,35 79,35 79,35 79,35 79,35 79,35 79,35 79,35 79,35 79,35 79,35 79,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 80,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 81,35 82,35 82,35 82,35 82,35 82,35 82,35 82,35 82,

# Plot of speed ramp

In [10]:
let sample_interval = 0.001_f32;  // in seconds
// Instanciate and configure the ramp
let mut r = Ramp::new(sample_interval,
    RampConstraints::default()
        .max_speed(2.0)
        .max_acceleration(1.0)
        .max_jerk(1.0));
let gp = GeneralizedPosition {position: 0.0, speed: 2.0, acceleration: 0.0 };
r.set(gp);
r.set_target_speed(-1.0);
println!("Ramp: {:?}", r);

Ramp: Ramp { sample_interval: 0.001, constraints: RampConstraints { max_speed: Some(2.0), max_acceleration: Some(1.0), max_jerk: Some(1.0) }, current: GeneralizedPosition { position: 0.0, speed: 2.0, acceleration: 0.0 }, state: [AccelerateTo(-1.0), AccelerateConst(2.0), AccelerateTo(0.0), SpeedKeep, SpeedZero, SpeedZero, SpeedZero, SpeedZero] }


In [11]:
let time_range = 5.0_f32;  // in seconds
let i_range = (time_range / sample_interval ) as usize;
// Run the ramp 
let gp = r.take(i_range).collect::<Vec<GeneralizedPosition>>();

In [12]:
// Extract position, speed and acceleration from generalized parameters for plotting
let s = gp.clone().iter().map(|x| x.speed).collect::<Vec<f32>>();
let a = gp.clone().iter().map(|x| x.acceleration).collect::<Vec<f32>>();

// Prepare the plot
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Position/speed/acceleration time diagram", ("Arial", 24).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.0f32..time_range, -1.0f32..2.0f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(s),
        &RED,
    )).unwrap()
        .label("Velocity")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (0..=i_range).map(|x| ( x as f32 * sample_interval)).zip(a),
        &BLUE,
    )).unwrap()
        .label("Acceleration")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));


    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    Ok(())
});

In [13]:
figure

Position/speed/acceleration time diagram
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 35,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 36,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 37,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 38,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 39,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 40,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 41,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 42,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 43,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 44,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 45,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 46,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 47,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 48,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 49,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 50,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 51,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 52,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 53,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 54,35 55,35 55,35 55,35 55,36 55,36 55,36 55,36 55,36 56,36 56,36 56,36 56,36 56,36 56,36 56,36 56,36 56,36 57,36 57,36 57,36 57,36 57,36 57,36 57,36 57,36 58,36 58,36 58,36 58,36 58,36 58,36 58,36 58,36 59,36 59,36 59,36 59,36 59,36 59,36 59,36 59,36 59,37 60,37 60,37 60,37 60,37 60,37 60,37 60,37 60,37 61,37 61,37 61,37 61,37 61,37 61,37 61,37 61,37 62,37 62,37 62,37 62,37 62,37 62,37 62,37 62,37 62,37 63,37 63,37 63,37 63,37 63,37 63,37 63,37 63,38 64,38 64,38 64,38 64,38 64,38 64,38 64,38 64,38 65,38 65,38 65,38 65,38 65,38 65,38 65,38 65,38 65,38 66,38 66,38 66,38 66,38 66,38 66,38 66,38 66,38 67,38 67,38 67,39 67,39 67,39 67,39 67,39 67,39 68,39 68,39 68,39 68,39 68,39 68,39 68,39 68,39 68,39 69,39 69,39 69,39 69,39 69,39 69,39 69,39 69,39 70,39 70,39 70,39 70,40 70,40 70,40 70,40 70,40 71,40 71,40 71,40 71,40 71,40 71,40 71,40 71,40 71,40 72,40 72,40 72,40 72,40 72,40 72,40 72,40 72,40 73,40 73,40 73,41 73,41 73,41 73,41 73,41 73,41 74,41 74,41 74,41 74,41 74,41 74,41 74,41 74,41 74,41 75,41 75,41 75,41 75,41 75,41 75,41 75,41 75,42 76,42 76,42 76,42 76,42 76,42 76,42 76,42 76,42 77,42 77,42 77,42 77,42 77,42 77,42 77,42 77,42 77,42 78,42 78,42 78,42 78,43 78,43 78,43 78,43 78,43 79,43 79,43 79,43 79,43 79,43 79,43 79,43 79,43 80,43 80,43 80,43 80,43 80,43 80,43 80,43 80,44 80,44 81,44 81,44 81,44 81,44 81,44 81,44 81,44 81,44 82,44 82,44 82,44 82,44 82,44 82,44 82,44 82,44 83,45 83,45 83,45 83,45 83,45 83,45 83,45 83,45 83,45 84,45 84,45 84,45 84,45 84,45 84,45 84,45 84,45 85,45 85,46 85,46 85,46 85,46 85,46 85,46 85,46 86,46 86,46 86,46 86,46 86,46 86,46 86,46 86,46 86,46 87,46 87,47 87,47 87,47 87,47 87,47 87,47 87,47 88,47 88,47 88,47 88,47 88,47 88,47 88,47 88,47 89,47 89,47 89,48 89,48 89,48 89,48 89,48 89,48 89,48 90,48 90,48 90,48 90,48 90,48 90,48 90,48 90,48 91,48 91,49 91,49 91,49 91,49 91,49 91,49 91,49 92,49 92,49 92,49 92,49 92,49 92,49 92,49 92,49 92,50 93,50 93,50 93,50 93,50 93,50 93,50 93,50 93,50 94,50 94,50 94,50 94,50 94,50 94,50 94,51 94,51 95,51 95,51 95,51 95,51 95,51 95,51 95,51 95,51 95,51 96,51 96,51 96,51 96,52 96,52 96,52 96,52 96,52 97,52 97,52 97,52 97,52 97,52 97,52 97,52 97,52 98,52 98,53 98,53 98,53 98,53 98,53 98,53 98,53 98,53 99,53 99,53 99,53 99,53 99,53 99,53 99,54 99,54 100,54 100,54 100,54 100,54 100,54 100,54 100,54 100,54 101,54 101,54 101,54 101,55 101,55 101,55 101,55 101,55 101,55 102